<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/01-nvidia-logo-horiz-500x200-2c50-d@2x.png" alt="NVIDIA Logo" style="width: 300px; height: auto;">


---


# Lab 2: Installing and Running AI Software

## Lab Overview

### Audience
The workbook is intended for technical training students.

### Objectives
In this practice, you will:

    ✓ Install the NVIDIA Container Toolkit on a server
    
    ✓ Run a container from NGC to train and recognize handwritten numbers

### Prerequisites and Guidelines
There are no prerequisites for this lab.

### Notice
Please follow the instructions below carefully to successfully complete the practice.
If you encounter technical issues, please contact the NVIDIA Networking Academy team:
nbu-academy-support@nvidia.com

---


## Practice 1: Install the Container Toolkit


Practice objectives:

In this exercise, you will install the NVIDIA Container Toolkit and check if the toolkit is installed and working properly by running a container.


### Task 1: Install the Container Toolkit


#### 1.1 - Configure a production repository:


In [4]:
!curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor --yes -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list > /dev/null

#### 1.2 - Update the packages list from the repository:


In [15]:
!sudo apt-get update

Get:1 file:/var/nvidia-driver-local-repo-ubuntu2404-580.105.08  InRelease [1572 B]
Get:1 file:/var/nvidia-driver-local-repo-ubuntu2404-580.105.08  InRelease [1572 B]
Hit:2 http://us-east4.gce.archive.ubuntu.com/ubuntu jammy InRelease            
Get:3 http://us-east4.gce.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://download.docker.com/linux/ubuntu jammy InRelease                 
Hit:5 https://repos.influxdata.com/debian stable InRelease                     
Hit:6 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease 
Get:7 http://us-east4.gce.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://apt.grafana.com stable InRelease                                 
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 http://us-east4.gce.archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3089 kB

#### 1.3 - Install the NVIDIA Container Toolkit packages:


In [14]:
!sudo apt-get install -y nvidia-container-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nvidia-container-toolkit is already the newest version (1.18.0-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


#### 1.4 - Configure a container runtime to enable support for NVIDIA GPUs via the NVIDIA Container Toolkit:


In [7]:
!sudo nvidia-ctk runtime configure

INFO[0000] Loading config from /etc/docker/daemon.json  
INFO[0000] Wrote updated config to /etc/docker/daemon.json 
INFO[0000] It is recommended that docker daemon be restarted. 


#### 1.5 - Restart the Docker service:


In [8]:
!sudo systemctl restart docker

---


### Task 2: Run the NVIDIA Container Toolkit


#### 2.1 - Run a container and check if the NVIDIA Container Toolkit operates as expected:


In [10]:
!sudo docker run --rm --runtime=nvidia --gpus all ubuntu nvidia-smi

Mon Nov 24 18:36:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:00:03.0 Off |                    0 |
| N/A   38C    P0             16W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### Does your output resemble the output as seen below?

```
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 NVL                Off |   00000000:B5:00.0 Off |                    0 |
| N/A   51C    P0            124W /  400W |       1MiB /  95830MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI              PID   Type   Process name                        GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|  No running processes found                                                             |
+-----------------------------------------------------------------------------------------+
```


#### Task 2 Questions:

    ✓ What command is executed in the container?
    
    ✓ What is the driver version in your output?
    
    ✓ What is the CUDA version in your output?


---


## Practice 2: Docker Container


Practice objectives:

In this exercise you will use a Docker container with an NVIDIA GPU to build a simple neural network to recognize a numeral digit from an image file.

This exercise assumes that the previous exercise "Install Container Toolkit" was completed successfully.


### Task 1: Install and Run PyTorch Docker Container


#### 1.1 - Run the following command to install a container:


In [11]:
!sudo docker pull nvcr.io/nvidia/pytorch:25.01-py3

25.01-py3: Pulling from nvidia/pytorch

b265507a: Pulling fs layer 
dd940fc6: Pulling fs layer 
b7e55b61: Pulling fs layer 
b700ef54: Pulling fs layer 
ff093155: Pulling fs layer 
fd5dddb1: Pulling fs layer 
63dd6b7e: Pulling fs layer 
4f040cbe: Pulling fs layer 
8ea39070: Pulling fs layer 
4e451322: Pulling fs layer 
80752b35: Pulling fs layer 
16b4865e: Pulling fs layer 
3b71b07b: Pulling fs layer 
59b21c0f: Pulling fs layer 
12f82f03: Pulling fs layer 
45aa1d7e: Pulling fs layer 
04ab24c5: Pulling fs layer 
a2f2d5bd: Pulling fs layer 
8363f368: Pulling fs layer 
7f42fa23: Pulling fs layer 
6e8d8318: Pulling fs layer 
3134baa1: Pulling fs layer 
77b797d2: Pulling fs layer 
abeffa5b: Pulling fs layer 
024db52a: Pulling fs layer 
d4d2d207: Pulling fs layer 
700ef54: Pulling fs layer 
6c1997d4: Pulling fs layer 
09fcf888: Pulling fs layer 
6d21c391: Pulling fs layer 
583d0756: Pulling fs layer 
65c05817: Pulling fs layer 
3986b8df: Pulling fs layer 
6aef40fb: Pulling fs layer 
fc731425:

#### 1.2 - Create a working folder for the project:


In [12]:
!mkdir -p docker_example

#### 1.3 - Before running the container, you may want to use "tmux" or "screen" to run monitoring commands on the host. Alternatively, you can open a new SSH connection to the host.

Run the container using the following command:

```bash
sudo docker run --gpus all --ipc=host -it --rm -v /home/student/AI_Infra/module8:/workspace nvcr.io/nvidia/pytorch:25.01-py3
```

**Note:** The directory `/home/student/AI_Infra/module8` on the host is mapped to the "workspace" directory in the container. This means that any file created in this directory on the host will appear in the container, and vice versa.


---


### Task 2: Prepare Program Code Files for Training


The following code will train a model to recognize a numeral in a 24x24 pixel image.

Navigate to the `~/AI_Infra/module8` directory, which contains the following files:

    ✓ train.py – Program that trains the AI
    
    ✓ test.py – Program that uses the trained AI
    
    ✓ 2.png, 7.png and 8.png – Image files used to test the AI

You can create a few 24x24 pixel images using software like MS Paint, writing a number in each image, or using files downloaded from GitHub. In our example, these are the images we used with significant magnification:


In [ ]:
!ls ~/AI_Infra/module8/

---


### Task 3: Train the Model


#### 3.1 - Before you start training your model, run nvidia-smi and make sure to remember or write down the following information:

    ✓ The GPU's memory usage before training begins
    
    ✓ The current GPU utilization
    
    ✓ The list of processes currently running on the GPU

Retain these details for later reference.


In [ ]:
!nvidia-smi

#### 3.2 - Run the training using the command "python train.py". During the training, several epochs will run:

**Note:** This command should be run from within the Docker container. You may need to execute the docker run command from Task 1.3 first.

Example output:

```
Train Epoch: 1 [0/60000 (0%)]   Loss: 2.303346             
Train Epoch: 1 [6400/60000 (11%)]       Loss: 0.147270     
Train Epoch: 1 [12800/60000 (21%)]      Loss: 0.200906     
Train Epoch: 1 [19200/60000 (32%)]      Loss: 0.102331     
Train Epoch: 1 [25600/60000 (43%)]      Loss: 0.226644     
Train Epoch: 1 [32000/60000 (53%)]      Loss: 0.024340     
Train Epoch: 1 [38400/60000 (64%)]      Loss: 0.034060     
Train Epoch: 1 [44800/60000 (75%)]      Loss: 0.005021     
Train Epoch: 1 [51200/60000 (85%)]      Loss: 0.002311     
Train Epoch: 1 [57600/60000 (96%)]      Loss: 0.022788     
                                                           
Test set: Average loss: 0.0406, Accuracy: 9878/10000 (99%)
```


#### 3.3 - While your model is training, run the nvidia-smi command again.

Compare the current output to what you observed before starting the training. Take note of the following:

    ✓ What was the GPU memory usage during training, compared to before you started?
    
    ✓ How did GPU utilization change during training?
    
    ✓ Which processes were running on the GPU during training?
    
    ✓ How much time did the model training take to complete?


In [ ]:
!nvidia-smi

---


### Task 4: Try the Trained Model


#### 4.1 - Use the test.py file to check if the trained model can recognize the numerals from the image files. 

By default, test.py will attempt to recognize the numeral in the file 7.png. To test a different file, provide its name as a parameter.

**Note:** These commands should be run from within the Docker container.

Example output:

```bash
root@20a0e58202b3:/workspace/AIDC-Professional# python test.py 
Processing file: 7.png
The predicted number is: 7
Device : cuda
Time : 3.02 seconds 
```


#### 4.2 - Test with a different image file:

```bash
root@20a0e58202b3:/workspace/AIDC-Professional# python test.py 8.png
Processing file: 8.png
The predicted number is: 7
Device : cuda
Time : 3.02 seconds
```


---


### Task 5: Compare the Load on GPU vs CPU


#### 5.1 - Use the command 'exit' to exit the running container.

Restart the container, but this time use the command:

```bash
sudo docker run --ipc=host -it --rm -v /home/student/AI_Infra/module8:/workspace nvcr.io/nvidia/pytorch:25.01-py3
```

**Note:** The `--gpus all` flag has been removed. The container will now run CPU-only, without utilizing the GPU.


#### 5.2 - The previously used files should still be in the container. Start training the model again.

**Question:** How much time did the model training take compared to GPU training?
